## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

------------------------------

#### **1. Inicializar y cargar el contexto spark**

In [ ]:
# Instalacion
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Importo un enviroment de java y spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Impoero e inicio findspark
import findspark
findspark.init()

# Inicio sesion del contexto spark
from pyspark import SparkContext

#Creo un objeto rdd para trabajar con el
sc = SparkContext.getOrCreate()

# Importo SparkSession desde pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Creacion de objeto llamado spark
spark = SparkSession.builder.master("local[*]").getOrCreate()

### CARGA DE BASES DE DATOS

1. Cargamos y visualización de los datos 

In [ ]:
# Desarrollo
data_file_desarrollo = "indicadores_desarrollo.csv"
raw_desarrollo = sc.textFile(data_file_desarrollo) # leemos el archivo 
raw_desarrollo.take(2)

['country_name,country_code,anho,GC.DOD.TOTL.CN,GC.DOD.TOTL.GD.ZS,SP.DYN.CBRT.IN',
 'Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259']

In [ ]:
# Deuda
data_file_deuda = "indicadores_deuda.csv"
raw_deuda = sc.textFile(data_file_deuda)
raw_deuda.take(2)

['country_name,country_code,DT.AMT.BLAT.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.MLAT.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.DIS.BLAT.CD,DT.DIS.DLXF.CD,DT.DIS.MLAT.CD,DT.DIS.OFFT.CD,DT.DIS.PCBK.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.INT.BLAT.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.MLAT.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD',
 'Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,']

2. Quitamos la cabecera

In [ ]:
# Desarrollo
cabecera =raw_desarrollo .first()
desarrollo_sin_cabecera = raw_desarrollo.filter(lambda x: x != cabecera)
desarrollo_sin_cabecera.take(2)

['Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259',
 'Albania,ALB,2015 [YR2015],1145500940000,79.8644448747259,12.197']

In [ ]:
# Deuda 
cabecera = raw_deuda.first()
deuda_sin_cabecera = raw_deuda.filter(lambda x: x != cabecera)
deuda_sin_cabecera.take(1)

['Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,']

3. Separamos por `,`

In [ ]:
# Desarrollo
desarrollo_pars = desarrollo_sin_cabecera.map(lambda x: x.split(","))
desarrollo_pars.take(2)

[['country_name',
  'country_code',
  'anho',
  'GC.DOD.TOTL.CN',
  'GC.DOD.TOTL.GD.ZS',
  'SP.DYN.CBRT.IN'],
 ['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259']]

In [ ]:
# Deuda
deuda_pars = deuda_sin_cabecera.map(lambda x: x.split(","))
deuda_pars.take(2)

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  ''],
 ['Albania',
  'ALB',
  '54250280.6',
  '790248675.2',
  '514185620',
  '182197616.7',
  '236447897.3',
  '0',
  '39445139.5',
  '170018.4',
  '39615157.9',
  '76050616.1',
  '317194512.5',
  '234321242.3',
  '310371858.4',
  '4542664.9',
  '2279989.2',
  '6822654.1',
  '13847333.6',
  '165602386.9',
  '87884000',
  '28101536.1',
  '41948869.7',
  '31030688.2',
  '4618504.3',
  '120324.7',
  '35769517.2']]

4. Paso a float los valores perdidos

In [ ]:
def tofloat (x):
  if x in ["" , " " , "nan" , "null"]:
    output = 0.0
  else:
    output = float(x)
  return output

5. Modificamos los nombres para que las filas tengan la misma longitud (cada celda en su sitio)

In [ ]:
# creamos una función
def parsed2(line,lon):
    line_splitted = line.split(",") # lo separo por ,
    if (len(line_splitted) == lon): #6 para desarrollo y 27 para deuda
        l = line_splitted
    else:
        l = line_splitted[2:] # los que no cumplan que sea = 2
        l.insert(0, line_splitted[0]+line_splitted[1]) # que me introduzca el nombre por delante
    return l

In [ ]:
# Desarrollo
desarrollo = desarrollo_sin_cabecera.map(lambda x: parsed2(x,6)).map(lambda x: len(x)).distinct() # todas las filas tienen la longitud de 6
desarrollo.take(2)

[6]

In [ ]:
# Deuda
deuda = deuda_sin_cabecera.map(lambda x: parsed2(x,27)).map(lambda x: len(x)).distinct()
deuda.take(2)

[27]

***

##### **2. Número de países distintos en cada dataset. Coinciden?**

* Indicadores Desarrollo

In [ ]:
desarrollo_pars.map(lambda x: x[0]).distinct().count() # tiene que salir 52

52

In [ ]:
print("El número de paises distintos de en la base de datos de indicadores de desarrollo es {} ".format(desarrollo_pars.map(lambda x: x[0]).distinct().count()))

El número de paises distintos de en la base de datos de indicadores de desarrollo es 52 


* Indicador de Deuda

In [ ]:
raw_deuda.map(lambda x: parsed2(x,27)).map(lambda x: x[0]).distinct().count()

125

In [ ]:
deuda_pars.map(lambda x: x[0]).distinct().count() # tiene que salir 124

123

In [ ]:
print("El número de paises distintos de en la base de datos de indicadores de deuda es {} ".format(deuda_pars_sin_cabecera.map(lambda x: x[0]).distinct().count()))

El número de paises distintos de en la base de datos de indicadores de deuda es 124 


##### **3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD**

In [ ]:
total_deuda = deuda_pars.map(lambda x: (x[0] , x[5])) # selecciono solo la columna 1 hasta la 6 
total_deuda.collect()

[('Afghanistan', '39107845'),
 ('Albania', '182197616.7'),
 ('Algeria', '0'),
 ('Angola', '124688691.9'),
 ('Armenia', '94331207.1'),
 ('Azerbaijan', '513623656.5'),
 ('Bangladesh', '1039564682.7'),
 ('Belarus', '595438826'),
 ('Belize', '29683326.5'),
 ('Benin', '91631639.5'),
 ('Bhutan', '17390420.5'),
 ('Bolivia', '357721211.5'),
 ('Bosnia and Herzegovina', '290170946.5'),
 ('Botswana', '125652344.5'),
 ('Brazil', '2514318741.6'),
 ('Bulgaria', '228993019.8'),
 ('Burkina Faso', '119510893'),
 ('Burundi', '16115111.8'),
 ('Cabo Verde', '39970124.9'),
 ('Cambodia', '72307001.8'),
 ('Cameroon', '152095968.1'),
 ('Central African Republic', '2241813.8'),
 ('Chad', '95738709.5'),
 ('China', '2615723714.1'),
 ('Colombia', '1100907553.6'),
 ('Comoros', '2182053.1'),
 ('"Congo', ''),
 ('"Congo', ''),
 ('Costa Rica', '272808260.7'),
 ("Cote d'Ivoire", '84629406.5'),
 ('Djibouti', '34254909.2'),
 ('Dominica', '11130355.8'),
 ('Dominican Republic', '254728306.1'),
 ('Ecuador', '578894793.9'),


##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

In [ ]:
#Cargo la libreria statistics para poder hacer la media 
from pyspark.mllib.stat import Statistics 

In [ ]:
# todo
filtro = deuda_pars.map(lambda x: (x[0],x[2],x[11],x[18])) 
filtro.collect()

[('Afghanistan', '61739336.9', '49114729.4', '39903620.1'),
 ('Albania', '54250280.6', '76050616.1', '13847333.6'),
 ('Algeria', '95188724.6', '10320772.2', '7680627.6'),
 ('Angola', '8473824016.3', '8838256901.1', '1005053965.1'),
 ('Armenia', '68968314.7', '174269846.7', '24094832'),
 ('Azerbaijan', '106369937', '84748079.6', '10085950'),
 ('Bangladesh', '386702219.8', '6141785637.5', '309079773.4'),
 ('Belarus', '1487363278.5', '2265336386.7', '630234914.9'),
 ('Belize', '23329498.4', '3581448.7', '3455608.4'),
 ('Benin', '26014505.8', '334698287.3', '13208408.6'),
 ('Bhutan', '189877869.8', '95819437.4', '184064524.8'),
 ('Bolivia', '67814879.9', '554128686.3', '32609012.4'),
 ('Bosnia and Herzegovina', '108704966.2', '114375048.3', '12605397.7'),
 ('Botswana', '7903563.5', '1316332.2', '884447.5'),
 ('Brazil', '2097380010.6', '1179125278.8', '333237826.9'),
 ('Bulgaria', '35764202.8', '', '6012379.2'),
 ('Burkina Faso', '27371336', '90915960.2', '7382485.5'),
 ('Burundi', '1046149

In [ ]:
indicador = filtro.filter(lambda x: not '"Congo' in x[0])\
                  .filter(lambda x: not '"Egypt' in x[0])\
                  .filter(lambda x: not '"Gambia' in x[0])\
                  .filter(lambda x: not '"Iran' in x[0])\
                  .filter(lambda x: not '"Macedonia' in x[0])\
                  .filter(lambda x: not 'Timor-Leste' in x[0])\
                  .filter(lambda x: not 'South Africa' in x[0])\
                  .filter(lambda x: not '"Yemen' in x[0])\
                  .filter(lambda x: not '"Venezuela' in x[0])\
                  .filter(lambda x: not 'Least developed countries: UN classification' in x[0])\
                  .filter(lambda x: not 'St. Lucia' in x[0])\
                  .filter(lambda x: not 'Somalia' in x[0])\
                  .filter(lambda x: not 'St. Lucia' in x[0])\
                  .filter(lambda x: not 'Tonga' in x[0])\
                  .filter(lambda x: not 'St. Vincent and the Grenadines' in x[0])\
                  .filter(lambda x: not 'Guinea-Bissau' in x[0]).persist() # para que quede guardado en la memoria
columna1 =indicador.map(lambda x: (tofloat(x[1]))).mean()
columna2= indicador.map(lambda x: (tofloat(x[2]))).mean()
columna3=indicador.filter(lambda x: x[3] != "").map(lambda x: (tofloat(x[3]))).mean()

In [ ]:
print("La media de la columna 1 es {}".format(round(columna1)))
print("La media de la columna 2 es {}".format(round(columna2)))
print("La media de la columna 3 es {}".format(round(columna3)))

La media de la columna 1 es 614452236
La media de la columna 2 es 991195020
La media de la columna 3 es 151600343


##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

In [ ]:
hdeuda=deuda_sin_cabecera.first()
# 27 es el numero de columnas
deuda_sin_cabecera.filter(lambda x: x != hdeuda).map(lambda x: parsed2(x,27)).map(lambda x: (x[0],x[3])).\
takeOrdered(20,lambda x: -float(x[1])) #-float para que me lo ordene de forma descendente y me quede el maximo el primero

[('China', '96218620835.7'),
 ('Brazil', '90041840304.1'),
 ('Russian Federation', '66589761833.5'),
 ('Turkey', '51555031005.8'),
 ('South Asia', '48756295898.2'),
 ('India', '31923507000.8'),
 ('Indonesia', '30916112653.8'),
 ('Kazakhstan', '27482093686.4'),
 ('Mexico', '25218503927'),
 ('Least developed countries: UN classification', '25197029299.4'),
 ('IDA only', '20483289208'),
 ('Romania', '14013783350.4'),
 ('Colombia', '11985674438.7'),
 ('Angola', '11067045628.1'),
 ('Cameroon', '10404814960.2'),
 ('"Venezuela RB"', '9878659207.2'),
 ('"Egypt Arab Rep."', '9692114176.9'),
 ('Lebanon', '9506919669.6'),
 ('South Africa', '9474257551.9'),
 ('Vietnam', '8873505909.2')]

##### 6. Pais con los datos informados todos los años.

In [ ]:
desarrollo_pars.map(lambda x: (x[0], x[2])).sortBy(lambda x: x[0], ascending = False).distinct().collect()

In [ ]:
pais =desarrollo_pars.filter(lambda x: "2017" in x[2]).map(lambda x: (x[0])).collect()
print("El país con datos de todos los años es" ,format(pais))

El país con datos de todos los años es ['Malawi']


##### 7. Media anual de los distintos indicadores de desarrollo

In [ ]:
from pyspark.sql import *
from pyspark.sql import SQLContext, functions as F, Row
from pyspark.sql.types import *
from pyspark.sql.types import StringType, IntegerType

In [ ]:
datos = pd.read_csv('indicadores_desarrollo.csv', sep =',', names=["country_name", "country_code","anho","GC_DOD_TOTL_CN","GC_DOD_TOTL_GD_ZS", "SP_DYN_CBRT_IN"], skiprows=1)

In [ ]:
df_desarrollo = sqlContext.createDataFrame(datos)
df_desarrollo.show(10)

+------------+------------+-------------+---------------+-------------------+------------------+
|country_name|country_code|         anho| GC_DOD_TOTL_CN|  GC_DOD_TOTL_GD_ZS|    SP_DYN_CBRT_IN|
+------------+------------+-------------+---------------+-------------------+------------------+
|     Albania|         ALB|2014 [YR2014]|   1.0230408E12|  73.32024691984292|            12.259|
|     Albania|         ALB|2015 [YR2015]|  1.14550094E12|   79.8644448747259|12.197000000000001|
|     Albania|         ALB|2016 [YR2016]|  1.18881449E12|  80.73557483969229|             12.08|
|   Australia|         AUS|2014 [YR2014]|        6.747E8|  0.042207528166503|              13.2|
|   Australia|         AUS|2015 [YR2015]|      7.62718E8| 0.0469480198522591|              12.9|
|   Australia|         AUS|2016 [YR2016]|      9.07555E8|0.05464848252016901|              12.9|
|Bahamas, The|         BHS|2014 [YR2014]|      5.15842E9|   47.2672793747079|             14.34|
|Bahamas, The|         BHS|201

In [ ]:
df_desarrollo.groupby('anho').agg(F.mean('GC_DOD_TOTL_CN')).show()

+-------------+--------------------+
|         anho| avg(GC_DOD_TOTL_CN)|
+-------------+--------------------+
|2016 [YR2016]|1.588138215480659...|
|2014 [YR2014]|1.034427587329532...|
|2017 [YR2017]| 2.23985861996269E12|
|2015 [YR2015]|1.318244039159025E14|
+-------------+--------------------+



In [ ]:
df_desarrollo.groupby('anho').agg(F.mean('GC_DOD_TOTL_GD_ZS')).show()

+-------------+----------------------+
|         anho|avg(GC_DOD_TOTL_GD_ZS)|
+-------------+----------------------+
|2016 [YR2016]|    58.796949624008356|
|2014 [YR2014]|    56.843914344384395|
|2017 [YR2017]|       48.659564238322|
|2015 [YR2015]|     58.36559834488034|
+-------------+----------------------+



In [ ]:
df_desarrollo.groupby('anho').agg(F.mean('SP_DYN_CBRT_IN')).show()

+-------------+-------------------+
|         anho|avg(SP_DYN_CBRT_IN)|
+-------------+-------------------+
|2016 [YR2016]| 16.144333333333332|
|2014 [YR2014]| 17.582620000000002|
|2017 [YR2017]|             34.593|
|2015 [YR2015]| 16.798477272727272|
+-------------+-------------------+



##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

In [ ]:
# funciones
from pyspark.sql.functions import col, avg, desc, sum

Creamos los dataframe para poder trabajar con ellos.

In [ ]:
# dataframe de desarrollo
df_desarrollo = pd.read_csv(data_file_desarrollo)
desarrollo_df = sqlContext.createDataFrame(df_desarrollo)
desarrollo_df.show(5)

+------------+------------+-------------+--------------+------------------+------------------+
|country_name|country_code|         anho|GC.DOD.TOTL.CN| GC.DOD.TOTL.GD.ZS|    SP.DYN.CBRT.IN|
+------------+------------+-------------+--------------+------------------+------------------+
|     Albania|         ALB|2014 [YR2014]|  1.0230408E12| 73.32024691984292|            12.259|
|     Albania|         ALB|2015 [YR2015]| 1.14550094E12|  79.8644448747259|12.197000000000001|
|     Albania|         ALB|2016 [YR2016]| 1.18881449E12| 80.73557483969229|             12.08|
|   Australia|         AUS|2014 [YR2014]|       6.747E8| 0.042207528166503|              13.2|
|   Australia|         AUS|2015 [YR2015]|     7.62718E8|0.0469480198522591|              12.9|
+------------+------------+-------------+--------------+------------------+------------------+
only showing top 5 rows



In [ ]:
#dataframe de deuda
df_deuda = pd.read_csv(data_file_deuda)
deuda_df = sqlContext.createDataFrame(df_deuda)
deuda_df.show(5)

+------------+------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
|country_name|country_code|DT.AMT.BLAT.CD|  DT.AMT.DLXF.CD|DT.AMT.DPNG.CD|DT.AMT.MLAT.CD|DT.AMT.OFFT.CD|DT.AMT.PBND.CD|DT.AMT.PCBK.CD|DT.AMT.PROP.CD|DT.AMT.PRVT.CD|DT.DIS.BLAT.CD|  DT.DIS.DLXF.CD|DT.DIS.MLAT.CD|DT.DIS.OFFT.CD|DT.DIS.PCBK.CD|DT.DIS.PROP.CD|DT.DIS.PRVT.CD|DT.INT.BLAT.CD|DT.INT.DLXF.CD|DT.INT.DPNG.CD|DT.INT.MLAT.CD|DT.INT.OFFT.CD|DT.INT.PBND.CD|DT.INT.PCBK.CD|DT.INT.PROP.CD|DT.INT.PRVT.CD|
+------------+------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------

A continuación los unimos

In [ ]:
union = desarrollo_df.join(other=deuda_df, on=["country_name"], how="inner")
union.show(5)

+------------+------------+-------------+-------------------+------------------+------------------+------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
|country_name|country_code|         anho|     GC.DOD.TOTL.CN| GC.DOD.TOTL.GD.ZS|    SP.DYN.CBRT.IN|country_code|DT.AMT.BLAT.CD|  DT.AMT.DLXF.CD|DT.AMT.DPNG.CD|DT.AMT.MLAT.CD|DT.AMT.OFFT.CD|DT.AMT.PBND.CD|DT.AMT.PCBK.CD|DT.AMT.PROP.CD|DT.AMT.PRVT.CD|DT.DIS.BLAT.CD|DT.DIS.DLXF.CD|DT.DIS.MLAT.CD|DT.DIS.OFFT.CD|DT.DIS.PCBK.CD|DT.DIS.PROP.CD|DT.DIS.PRVT.CD|DT.INT.BLAT.CD|  DT.INT.DLXF.CD|DT.INT.DPNG.CD|DT.INT.MLAT.CD|DT.INT.OFFT.CD|DT.INT.PBND.CD|DT.INT.PCBK.CD|DT.INT.PROP.CD|DT.INT.PRVT.CD|
+------------+--

In [ ]:
# Para renombrar (porque nos da probelmas) las columnas utilizaremos la función withColumnRenamned()
df = union.withColumnRenamed("anho", "year") \
    .withColumnRenamed("DT.AMT.MLAT.CD", "deuda") \
    .withColumnRenamed("SP.DYN.CBRT.IN", "birthdate")

In [ ]:
colum = df.select("country_name","deuda","birthdate")
colum.show()


+------------+-------------+------------------+
|country_name|        deuda|         birthdate|
+------------+-------------+------------------+
| Philippines|7.390564273E8|            22.935|
|       Tonga|    2791253.2|            25.756|
|       Tonga|    2791253.2|            25.354|
|       Tonga|    2791253.2|24.976999999999997|
|      Turkey|2.932698678E9|             16.94|
|      Turkey|2.932698678E9|            16.718|
|      Turkey|2.932698678E9|            16.494|
|      Malawi| 3.56598314E7|            36.629|
|      Malawi| 3.56598314E7|            35.839|
|      Malawi| 3.56598314E7|35.163000000000004|
|      Malawi| 3.56598314E7|            34.593|
|      Jordan|2.704987155E8|25.214000000000002|
|      Jordan|2.704987155E8|            23.396|
|   Sri Lanka|4.497719611E8|            16.849|
|   Sri Lanka|4.497719611E8|            16.609|
|     Albania|1.821976167E8|            12.259|
|     Albania|1.821976167E8|12.197000000000001|
|     Albania|1.821976167E8|            

In [ ]:
# agrupamos
total = colum.groupBy("country_name", "deuda").agg(avg("birthdate"))
total.show()

+------------------+--------------+------------------+
|      country_name|         deuda|    avg(birthdate)|
+------------------+--------------+------------------+
|       Philippines| 7.390564273E8|            22.935|
|             Tonga|     2791253.2| 25.36233333333333|
|            Turkey| 2.932698678E9|16.717333333333332|
|            Malawi|  3.56598314E7|            35.556|
|            Jordan| 2.704987155E8|            24.305|
|         Sri Lanka| 4.497719611E8|            16.729|
|           Albania| 1.821976167E8|12.178666666666667|
|              Peru|   5.5363409E8|18.564666666666668|
|         St. Lucia|  1.50192163E7|            12.502|
|           Belarus|  5.95438826E8|12.466666666666667|
|Russian Federation|    1.416695E8|13.166666666666666|
|          Thailand|  3.90715715E7|           10.9175|
|            Bhutan|  1.73904205E7|            17.791|
|           Ukraine|1.0724728303E9|              10.6|
|           Georgia| 1.440624015E8|            14.056|
|         

In [ ]:
# seleccionamos las 10
total = total.sort("avg(birthdate)").limit(10)
total.show()

+--------------------+--------------+------------------+
|        country_name|         deuda|    avg(birthdate)|
+--------------------+--------------+------------------+
|             Ukraine|1.0724728303E9|              10.6|
|             Moldova| 1.358428655E8|10.604333333333335|
|            Thailand|  3.90715715E7|           10.9175|
|             Albania| 1.821976167E8|12.178666666666667|
|             Belarus|  5.95438826E8|12.466666666666667|
|           St. Lucia|  1.50192163E7|            12.502|
|  Russian Federation|    1.416695E8|13.166666666666666|
|             Georgia| 1.440624015E8|            14.056|
|              Brazil|2.5143187416E9|14.467666666666668|
|St. Vincent and t...|  2.16156007E7|            15.011|
+--------------------+--------------+------------------+



In [ ]:
# sumamos todo
total.select(sum("deuda")).show()

+-------------------+
|         sum(deuda)|
+-------------------+
|4.861709170099999E9|
+-------------------+



##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?

In [ ]:
# funcion para hacer correlacion 
from pyspark.ml.stat import Correlation

In [ ]:
# renombro  "DT.AMT.DLXF.CD" porque me da problemas 
renom = df.withColumnRenamed("DT.AMT.DLXF.CD", "deuda_dlxf")

In [ ]:
# seleccionamos las variables que necesitamos
select = renom.select("birthdate","deuda_dlxf")
select.show()

+------------------+----------------+
|         birthdate|      deuda_dlxf|
+------------------+----------------+
|            22.935|  6.4791709177E9|
|            25.756|    1.03699106E7|
|            25.354|    1.03699106E7|
|24.976999999999997|    1.03699106E7|
|             16.94|5.15550310058E10|
|            16.718|5.15550310058E10|
|            16.494|5.15550310058E10|
|            36.629|    5.74223731E7|
|            35.839|    5.74223731E7|
|35.163000000000004|    5.74223731E7|
|            34.593|    5.74223731E7|
|25.214000000000002|  1.9903434792E9|
|            23.396|  1.9903434792E9|
|            16.849|  5.7614892559E9|
|            16.609|  5.7614892559E9|
|            12.259|   7.902486752E8|
|12.197000000000001|   7.902486752E8|
|             12.08|   7.902486752E8|
|            18.827|  5.8812599747E9|
|            18.544|  5.8812599747E9|
+------------------+----------------+
only showing top 20 rows



In [ ]:
# correlacion 
select.stat.corr("deuda_dlxf", "birthdate")

-0.28663478114959196

--------------------

In [ ]:
# Finalizo el contexto Spark
sc.stop()